In [ ]:
!pip install opencv-python-headless tensorflow torch torchvision

In [20]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

# Load the dataset
data_train_path = 'data/train/DataTrain.csv'
image_folder_train = 'data/train'
train_data = pd.read_csv(data_train_path, sep=';')
train_data['ImagePath'] = train_data['NameofFile'].apply(lambda x: os.path.join(image_folder_train, x))

train_data

In [2]:
def enhance_brightness_contrast_with_clahe(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the mean brightness of the image
    mean_brightness = np.mean(gray)
    # If the image has low brightness, enhance contrast
    if mean_brightness < 128:
        # Convert the image to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        # Split the LAB image into L, A, and B channels
        l, a, b = cv2.split(lab)
        # Apply CLAHE to the L channel
        clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        # Convert back to BGR color space
        enhanced_image = cv2.cvtColor(limg, cv2.COLOR_Lab2BGR)
        return enhanced_image
    else:
        # Enhance contrast using histogram equalization
        gray_equalized = cv2.equalizeHist(gray)
        # Convert back to BGR color space
        enhanced_image = cv2.cvtColor(gray_equalized, cv2.COLOR_GRAY2BGR)
        return enhanced_image

In [3]:
def denoise_image(image):
    # Apply Non-Local Means Denoising
    image = cv2.imread(image_path)
    # Apply Non-Local Means Denoising
    return cv2.fastNlMeansDenoising(image, None, 30, 7, 50)

In [4]:
from scipy.signal import convolve2d

def wiener_deconvolution(image_path, kernel_size, sigma):
    image = cv2.imread(image_path)
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size ** 2)
    deblurred_image = cv2.filter2D(image, -1, kernel)
    return deblurred_image


In [27]:
def sharpen_image(image, iterations=5):
    # Apply the Unsharp Mask filter
    blurred = cv2.GaussianBlur(image, (0, 0), 3)
    sharpened = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)
    
    # Apply multiple iterations of sharpening
    for _ in range(iterations - 1):
        sharpened = cv2.addWeighted(sharpened, 1.5, blurred, -0.5, 0)
    
    return sharpened

In [62]:
for image_path in train_data['ImagePath']:
    # Enhance brightness and contrast
    enhanced_image = enhance_brightness_contrast_with_clahe(image_path)
    # Convert to grayscale
    gray_image = cv2.cvtColor(enhanced_image, cv2.COLOR_BGR2GRAY)
    # Denoise the image
    resized_image = cv2.resize(gray_image, (300,100))
    sharpened_image = sharpen_image(resized_image)
    # # Define the kernel for deblurring
    # # Deblur the image
    # denoised_image = denoise_image(image_path)
    # #save the image
    cv2.imwrite(image_path, sharpened_image)


In [64]:
for image_path in train_data['ImagePath']:
    denoised_image=denoise_image(image)
    cv2.imwrite(image_path, denoised_image)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def sharpen_and_denoise_image(image_path):
    # Read the image
    image = cv2.imread(image_path)


    # Sharpen the image (use a stronger kernel)
    kernel = np.array([[0, -2, 0], [-2, 9, -2], [0, -2, 0]])
    sharpened_image = cv2.filter2D(image, -1, kernel)

    # Convert the image from BGR to RGB
    sharpened_image = cv2.cvtColor(sharpened_image, cv2.COLOR_BGR2RGB)

   # Save the processed image back to the original file
    cv2.imwrite(image_path, sharpened_image)

# Process the first five images
for image_path in train_data['ImagePath'][5:10]:
    sharpen_and_denoise_image(image_path)
    


In [ ]:
def draw_license_plate_box(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use thresholding to highlight the license plate
    _, thresh = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding box around the detected license plate
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Filter out very small and very large boxes
        if 1000 > w * h > 100:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Convert the image from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Process the first five images
for image_path in train_data['ImagePath'][:10]:
    draw_license_plate_box(image_path)

In [ ]:
!pip install easyocr


In [ ]:
import easyocr
import matplotlib.pyplot as plt
import cv2

# Initialize the easyocr reader
reader = easyocr.Reader(lang_list=['en'])

def recognize_license_plate(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Convert the image from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Use easyocr to recognize text
    results = reader.readtext(image)

    # Draw the bounding boxes on the image
    for (bbox, text, prob) in results:
        # Unpack the bounding box
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))
        
        # Draw the bounding box and text
        cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 2)
        cv2.putText(image, text, (top_left[0], top_left[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        
        # Print the recognized text
        print(f"Recognized text: {text}")

    # Convert the image from RGB to BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

# Process the first five images
for image_path in train_data['ImagePath'][:5]:
    recognize_license_plate(image_path)
